# Build a Chatbot

In [4]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import warnings

warnings.filterwarnings("ignore")

# Load environment variables from .env file
load_dotenv()

# Create LLM Model instance and invoke it directly
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob, I'm Gemini, a multi-modal AI model, developed by Google. Is there anything I can assist you with today?", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-141b4ca3-93b1-401b-b9e3-96e3c84aebc2-0', usage_metadata={'input_tokens': 7, 'output_tokens': 29, 'total_tokens': 36})

In [5]:
# Chat models do not have any concept of state, they don't keep track of past messages unless explicitly coded to do so
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='I do not have enough information to determine your name.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8ac6c487-a399-4d23-9bc4-3341230599c9-0', usage_metadata={'input_tokens': 7, 'output_tokens': 11, 'total_tokens': 18})

In [6]:
# To get around this, we need to pass the entire conversation history into the model.
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You told me your name is Bob. Is that correct?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f9281ba3-d7f9-4bc6-ae6f-dc364b0463c4-0', usage_metadata={'input_tokens': 25, 'output_tokens': 12, 'total_tokens': 37})

In [7]:
# We can use a Message History class to wrap our model and make it stateful. 
# This will keep track of inputs and outputs of the model, and store them in some datastore.
# BaseChatMessageHistory provides a framework for creating chat message history implementations,
# while InMemoryChatMessageHistory offers a simple and efficient way to store chat history in memory.
# RunnableWithMessageHistory essentially wraps an LLM and automatically handles the integration of a chat message history,
# such as InMemoryChatMessageHistory. 
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

# We now need to create a config that we pass into the runnable every time. 
# This config contains information that is not part of the input directly, but is still useful.
config = {"configurable": {"session_id": "abc2"}}

In [8]:
# Start conversation
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Rishil")],
    config=config,
)

response.content

"Hi Rishil, it's nice to meet you! I'm Gemini, a multi-modal AI language model developed by Google. I can help you with a wide range of topics and tasks, from answering your questions to generating text and translating languages. Feel free to ask me anything you'd like, and I'll do my best to assist you."

In [9]:
# Check if LLM remembers your name
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You told me your name is Rishil. Is that correct?'

In [10]:
# Every new session id refers to a new conversation, means we can start a fresh new conversation with a new session id
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I do not have access to your personal information and cannot tell you your name.'

In [12]:
# We can always go back to an older session and continue the conversation
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You still have not told me your name, so I do not know what it is. Would you like to tell me your name?'

In [13]:
# Prompt templates help turn raw user information into a format that the LLM can work with
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

# Now we invoke the chain using a dictionary
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response.content

'Hi Bob! How can I help you today?'

In [ ]:
# Use this new chain with Chat prompt and also save message history
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
# Start a new conversation
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

"Hello Jim! It's nice to meet you. How can I assist you today? Do you have any questions or tasks you'd like me to help you with?"

In [14]:
# Check if model retains name from message history
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You have not told me your name yet, so I do not know what it is. Would you like to tell me your name?'

In [15]:
# A more complicated prompt to get response from LLM in mentioned language
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
# Get response in mentioned language
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola! Soy Bob, tu asistente. Haré todo lo posible para responder a tus preguntas en español.'

In [20]:
# Keep message history for chatbot that responds in specific language
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! Me alegro de poder ayudarte. ¡Sólo pregunta!'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Japanese"},
    config=config,
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte hoy?'

In [28]:
# To limit the history that an LLM should maintain so that it does not go to infinity, we need to trim the history
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[HumanMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [29]:
# Adding trimmer in LLM chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't know your name, as I am just an AI chatbot assistant and do not have access to personal information."

In [30]:
# If we ask about information that is within the last few messages, it remembers
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'2 + 2'

In [31]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [32]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'bob'

In [33]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'2 + 2'

In [34]:
# Streaming allows to continuously get output from the LLM in chunks as LLMs take longer to give the complete output
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

What do you call a boomerang that doesn’t come back? A stick|.|